# Search Repeat Cycle of Satellite from its TLE

In [1]:
import ephem

In [2]:
# Two Line Element
line1 = "WORLDVIEW-1 (WV-1)"
line2 = "1 32060U 07041A   18258.02295190  .00000790  00000-0  35109-4 0  9993"
line3 = "2 32060  97.3879  16.4069 0002228  57.6683  54.8987 15.24397549611548"
satellite = ephem.readtle(line1, line2, line3)

In [3]:
# Get Ephemeris Data
def getEphem(satellite, date):
  satellite.compute(date)
  latitude  = satellite.sublat / ephem.degree
  longitude = satellite.sublong / ephem.degree
  hight     = satellite.elevation
  mmotion   = satellite._n
  obperiod  = 1 / mmotion
  return (latitude, longitude, hight, mmotion, obperiod)

In [4]:
# Search date with Latitude = 0
# date: latitude must be near 0 at specified date
# eps : small enough value
def lat0date(satellite, date, eps):
  d1 = date
  satellite.compute(d1)
  l1  = satellite.sublat / ephem.degree
 
  d2 = d1 + eps
  satellite.compute(d2)
  l2  =satellite.sublat / ephem.degree

  a = (l2 - l1) / eps
  b = l1 - a * d1

  dt   = -b / a
  
  return dt

In [5]:
# Check Latitude is almost 0
# Latitude must be near 0 at specified date
dt  = ephem.Date("2018-09-18 10:46:01")
dt0 = dt

(latitude, longitude, hight, mmotion, obperiod) = getEphem(satellite, dt)
print("Lat(+N):%.4f, Long(+E):%.4f, Hight(m):%.1f, MeanMotion(RevPerDay):%.8f, OrbitalPeriod(days):%.8f" % (latitude, longitude, hight, mmotion, obperiod))

Lat(+N):-0.1840, Long(+E):40.9467, Hight(m):496775.2, MeanMotion(RevPerDay):15.24397549, OrbitalPeriod(days):0.06559969


In [6]:
# Calculate time with Latitude = 0
ep = obperiod / 360 / 10
dt = lat0date(satellite, dt0, ep)
dt00 = dt
(latitude, longitude, hight, mmotion, obperiod) = getEphem(satellite, dt)
print("Lat(+N):%.4f, Long(+E):%.4f, Hight(m):%.1f, MeanMotion(RevPerDay):%.8f, OrbitalPeriod(days):%.8f" % (latitude, longitude, hight, mmotion, obperiod))

latitude0  = latitude
longitude0 = longitude
mmotion0   = mmotion
obperiod0  = obperiod

Lat(+N):0.0000, Long(+E):40.9827, Hight(m):496747.4, MeanMotion(RevPerDay):15.24397549, OrbitalPeriod(days):0.06559969


In [7]:
# Search Repeat Cycle

maxlat   = 1.0 # max error of latitude[deg]
maxlong  = 1.0 # max error of longitude[deg]
maxdays  = 190 # max days of search

latlen   = 40009 # Circumference - meridional [Km]
longlen  = 40075 # Circumference - quatorial  [Km]

dt = dt00
print("Repeat Cycle of " + satellite.name)
print("                                         Lat(+N) diff[deg]   diff[Km] |   Long(+E)  diff[deg]   diff[Km]")
for d in range(int(maxdays * mmotion0)):
  (latitude, longitude, _, _, _) = getEphem(satellite, dt)
  difflat   = latitude  - latitude0
  difflong  = longitude - longitude0
  
  if abs(difflat) < maxlat and abs(difflong) <maxlong:
    dtstr= str(ephem.localtime(ephem.Date(dt)))[0:19]
    days = dt - dt00
    difflatlen  = difflat  / 360 * latlen
    difflonglen = difflong / 360 * longlen
    print("[%s = %6.2f(days)] %10.4f %+10.4f %+10.4f | %10.4f %+10.4f %+10.4f" % (dtstr, days, latitude, difflat, difflatlen, longitude, difflong, difflonglen))

  # update dt
  dt = lat0date(satellite, dt + obperiod, ep)

Repeat Cycle of WORLDVIEW-1 (WV-1)
                                         Lat(+N) diff[deg]   diff[Km] |   Long(+E)  diff[deg]   diff[Km]
[2018-09-18 19:45:58 =   0.00(days)]     0.0000    +0.0000    +0.0000 |    40.9827    +0.0000    +0.0000
[2018-10-05 19:47:24 =  17.00(days)]     0.0000    -0.0000    -0.0001 |    40.6255    -0.3573   -39.7726
[2018-10-18 19:42:38 =  30.00(days)]     0.0000    -0.0000    -0.0001 |    41.8222    +0.8395   +93.4510
[2018-10-22 19:48:23 =  34.00(days)]     0.0000    -0.0000    -0.0001 |    40.3868    -0.5959   -66.3356
[2018-11-04 19:43:14 =  47.00(days)]     0.0000    -0.0000    -0.0001 |    41.6753    +0.6925   +77.0932
[2018-11-08 19:48:52 =  51.00(days)]     0.0000    -0.0000    -0.0001 |    40.2682    -0.7146   -79.5437
[2018-11-21 19:43:22 =  64.00(days)]     0.0000    -0.0000    -0.0001 |    41.6482    +0.6655   +74.0808
[2018-11-25 19:48:54 =  68.00(days)]     0.0000    -0.0000    -0.0001 |    40.2692    -0.7135   -79.4312
[2018-12-08 19:43:02